In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/New_datasets/gnq

#Setting up libraries

In [ ]:
# pip installing all the libraries
%%capture
!pip install jax
!pip install flax
!pip install sentencepiece
!pip install transformers
!pip install -U sentence-transformers



In [ ]:
#All the imports
from google.colab import output
import pandas as pd
import pickle
import jax
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import jax.numpy as jnp
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
import torch
import urllib3
import requests
import re
import pickle
from googlesearch import search
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import timeit
import nltk
import textwrap
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
from transformers import FlaxBigBirdForQuestionAnswering, BigBirdTokenizer
# import gradio as gr
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import urlparse
nltk.download('punkt')
from sklearn.cluster import KMeans



#Utilites and Functions


In [ ]:

def combine_strs(list_of_str):
  ans_list = []
  i=0
  while i < (len(list_of_str)):
    j = i
    temp_str =""
    while(j<len(list_of_str) and len(temp_str.split(" "))<256):
      temp_str+=list_of_str[j]+" "
      # print(temp_str)
      j+=1
    ans_list.append(temp_str)
    i=j
  return ans_list

In [ ]:
#Word Wrap
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
#utils
class deli:
  @staticmethod
  def deli_1():
    print("_______________ this is a delimiter _______________")
  @staticmethod
  def deli_2():
    print("_______________________________________________________________________________________________")
  @staticmethod
  def deli_3():
    print("========================================================================")
def list_to_str(some_list):
  return " ".join(some_list)
def split_into_sent(content):
  a_list = nltk.tokenize.sent_tokenize(content)
  return a_list

def harmonic_mean(a):
  b=a[1]
  a=a[0]
  if a==0 or b==0:
    return 0
  else:
    return 2*a*b/(a+b)
import re
CLEANR = re.compile('<.*?>')
spaces = re.compile(' +')
# delimiters = re.compile('')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, ' ', raw_html)
  cleantext = re.sub(spaces, ' ', cleantext)
  return cleantext

def simplify(st):
  temp = cleanhtml(st)
  temp = temp.replace('\n',' ')
  temp = temp.replace('\t',' ')
  return temp


def printw(f):
  wrapper = textwrap.TextWrapper(width=180)
  
  word_list = wrapper.wrap(text=f)
  
# Print each line.
  for element in word_list:
    print(element)

def make_histogram(y):
  hist= {}
  for i in y:
    hist[i] = hist.get(i,0)+1
  return hist
def combine_strs(list_of_str):
  ans_list = []
  i=0
  while i < (len(list_of_str)):
    j = i
    temp_str =""
    while(j<len(list_of_str) and len(temp_str.split(" "))<256):
      temp_str+=list_of_str[j]+" "
      # print(temp_str)
      j+=1
    ans_list.append(temp_str)
    i=j
  return ans_list



#Define Models


In [ ]:
%%capture
FLAX_MODEL_ID = "vasudevgupta/flax-bigbird-natural-questions"
model_qa = FlaxBigBirdForQuestionAnswering.from_pretrained(FLAX_MODEL_ID, block_size=64, num_random_blocks=3)
tokenizer_qa = BigBirdTokenizer.from_pretrained(FLAX_MODEL_ID)

@jax.jit
def forward(*args, **kwargs):
  return model_qa(*args, **kwargs)

def get_answer(question, context):

    encoding = tokenizer(question, context, return_tensors="jax", max_length=4096, padding="max_length", truncation=True)
    start_scores, end_scores = forward(**encoding).to_tuple()

    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

    answer_tokens = all_tokens[jnp.argmax(start_scores): jnp.argmax(end_scores)+1]
    answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

    return answer
def run_code():
  question = input("Type your question here : ")
  context = input("Paste the context here : ") 
  ans = get_answer(question, context)
  print("Lets see :- ")
  print(ans)


In [ ]:
def get_answer_1(question, context):
  # print("I am in this function")
  if(context==" this is a meaningless content due to repetition and timeout as well"):
    return ("No Answer due to meaningless content due to repetition and timeout as well",(0,0))

  encoding = tokenizer_qa(question, context, return_tensors="jax", max_length=4096, padding="max_length", truncation=False)
  start_scores, end_scores = forward(**encoding).to_tuple()
  # print(start_scores.shape, end_scores)
  # Let's take the most likely token using `argmax` and retrieve the answer
  all_tokens = tokenizer_qa.convert_ids_to_tokens(encoding["input_ids"][0].tolist())
  # print(all_tokens)
  g = (jnp.max(start_scores), jnp.max(end_scores))
  start_token = jnp.argmax(start_scores)
  end_token =  jnp.argmax(end_scores)
  answer_tokens = all_tokens[start_token: end_token+1]
  answer = tokenizer_qa.decode(tokenizer_qa.convert_tokens_to_ids(answer_tokens))
  answer=answer.replace("[SEP]", "")
  answer=answer.replace("[CLS]", "")
  # print(start_token, end_token)
  if(answer==""):
    # print("Stuck here")
    # print(end_scores.shape)
    end_token_ = jax.numpy.argsort((end_scores[0]))
    # print(end_token_, end_scores, "Sanity Check")
    # pprint(list(end_scores[0]))
    if(len(end_token_)==1):
      answer = "No Meaningful Answer due to lack of content only 1 word"
    else:
      # print("I am here")
      end_token = end_token_[-2]
      g = (g[0],end_scores[0][end_token_[-2]])
      answer_tokens = all_tokens[start_token: end_token+1]
      answer = tokenizer_qa.decode(tokenizer_qa.convert_tokens_to_ids(answer_tokens))
      answer=answer.replace("[SEP]", "")
      answer=answer.replace("[CLS]", "")
      
    # print(g, ((start_scores), (end_scores)))
  answer=answer.replace("⁇", "")
  if(answer =="" or answer ==" "):
    g = (1 if g[0]==0 else g[0],0)
  # print("I am leaving this function")
  return (answer,g)

#Crawler


In [ ]:
def google_search(q, n, ls):
  for j in search(q, tld="com", num=10, stop=10, pause=2):
      ls.append(j)


def scraper(query_list,n_links=10):
  df = pd.DataFrame()
  count_F=0
  for i,q in enumerate(query_list):
    print("i->", i)
    row =dict()
    q = q[0]
    row['query'] = q
    ls = []
    unique_url = set()
    print(q[0])
    google_search( q, n_links ,ls)
    # print(ls)
    for j,url in enumerate(ls):
      print("\tj->", j)
      # webpage = requests.get(url)
      # webpage= webpage.content.encode('utf-8')
      domain = urlparse(url).netloc
      if domain in unique_url:
        print("same domain explored before")
        pass
      unique_url.add(domain)
      try:
          http_pool = urllib3.connection_from_url(url)
          r = http_pool.urlopen('GET',url)
          content= r.data.decode('utf-8')

      except:
          try:
              webp = requests.get(url)
              content = webp.content.encode('utf-8')
          except:
              content= "This is a meaningless content   "
              print("F")
              count_F+=1
              pass

      
      
      soup_obj = BeautifulSoup(content, 'lxml')

      for script in soup_obj(["script", "style"]):
          script.extract()

      p1  = []
      for para in soup_obj.find_all(["p","li"]):
        p1.append(para.get_text()+" ")
      row[str(j)+"_Content"] = "\n".join(p1)
      row[str(j)+"_Link"] = url
    df = df.append(row, ignore_index=True)
  print(count_F)
  return df


In [ ]:
#google search api + 
def scraper(query_list,n_links=10):
  df = pd.DataFrame()
  for i,q in enumerate(query_list):
    print("i", i)
    row =dict()
    row['query'] = q[0] 
    ls = []
    
    google_search(q,n_links, ls)
    for j,url in enumerate(ls):
      print("\tj", j)
      webpage = requests.get(url)
      # webpage= webpage.content.encode('utf-8')
      if (i,j) in [(0,8)]:
        try:
          decoded_webpage = webpage.content.decode('utf-8')
        except:
          print(i,j,"checked cases")
          pass
      else:
        decoded_webpage = webpage.content.decode('utf-8')

      soup_obj = BeautifulSoup(decoded_webpage, 'lxml')

      for script in soup_obj(["script", "style"]):
          script.extract()   

      p1  = []
      for para in soup_obj.find_all(["p","li"]):
        p1.append(para.get_text()+" ")
      row[str(j)+"_Content"] = "\n".join(p1)
      row[str(j)+"_Link"] = url
    df = df.append(row, ignore_index=True)
  return df





#Document Shortening Algorithm


##TFIDF + Cosine similarity + window size =2

In [ ]:
#TFIDF + Cosine similarity + window size =2
# def select_sent(query,sent_list,thresh=0.2):
#   vect = TfidfVectorizer(min_df=1, stop_words="english")
#   try:
#     tfidf = vect.fit(sent_list)
#   except:
#     return [(i,k) for i,k in enumerate(sent_list)]

#   ans=[]
#   for i,sent in enumerate(sent_list):
#     s = [query, sent]
#     tfidf_1 = tfidf.transform(s)
#     pairwise_similarity = tfidf_1 * tfidf_1.T
#     simma = ( pairwise_similarity.toarray())[0,1]
#     if(simma>thresh):
#       ans.append((i,sent))
#   return ans
  
# def list2para(ans_list,sent_list,window_size=2):
  
#   d= set()
#   len_sent= len(sent_list)
#   for i,k in ((ans_list)):
#     for j in range(-window_size,window_size+1):
#       if(i+j>=0 and i+j<len_sent):
#         d.add((i+j,sent_list[i+j]))
#   d = list(d)
#   d =sorted(d)
#   return list_to_str([j[1] for j in list(d)])


# def answer_2(df):
#   df=df.fillna("")
#   list_1=[]
#   for i, k in df.iterrows():
#     q = k['query']
#     print(i)
#     list_temp=[]
#     for j in range( 10):
#       c = k[str(j)+"_Content"]
#       # c = make_tuple()
#       c = simplify(c)


#       sent_list = split_into_sent(c)
      
#       ans_list = select_sent(q,sent_list,thresh= 0.15)

#       l2p = list2para(ans_list,sent_list)
#       ans= get_answer_1(q,l2p)
#       list_temp.append((harmonic_mean(ans[1]),ans[0],j))
#     list_temp_1 = sorted(list_temp)
#     list_temp_1.reverse()
#     list_temp_temp = [(ii[1],ii[2]) for ii in list_temp_1]
#     list_temp = [q,k['ans']]
#     list_temp.extend(list_temp_temp)
#     list_temp=list_temp[:7]
#     list_1.append(list_temp)
#   return list_1

##Passage Ranking with MS Marco


In [ ]:
model_n='cross-encoder/ms-marco-TinyBERT-L-2'
model_pr = AutoModelForSequenceClassification.from_pretrained(model_n)
tokenizer_pr = AutoTokenizer.from_pretrained(model_n)

In [ ]:
def score_for_one(q, list_of_passages):
  
  features=tokenizer_pr([q],list_of_passages,padding =True, truncation = True, return_tensors='pt')

  model_pr.eval()
  with torch.no_grad():
      scores = model_pr(**features).logits
      return scores

In [ ]:
def rank_passages_1(df_c,top_k=9):
  df_c=df_c.fillna("")
  df_ans = pd.DataFrame()
  len_array=[]
  for i, k in df_c.iterrows():
    row_dict={}
    for j in range(10):
      print(i,j)
      passage_ranker=[]
      q=df_c['query'].iloc[i]

      content= df_c[str(j) +"_Content"].iloc[i]
      printw("Question "+str(i) +":" + q)
      printw("Content "+str(j) +":" + df_c[str(j) +"_Link"].iloc[i])
      if(content==""):
        row_dict[str(j)] = "This is Empty Content"
        continue
      cont_l= content.split("\n")
      cont_l_1=[]
      for ii in cont_l:
        if ii == "":
          continue
        if len(ii.split(" "))>450:
          print("I am a problem, u shud see me", i,j)
        
        cont_l_1.append(ii)
      # cont_l_2 = combine_strs(cont_l_1)
      # cont_l_1=cont_l_2
      print(len(cont_l_1))
      len_array_1 = [len(i.split(" ")) for i in cont_l_1]
      len_array.extend(len_array_1)

      scores = []
      
      row_dict['query']=q
      for c,ii in enumerate(cont_l_1):
        score = score_for_one(q, [ii])
        scores.append((score, c))
      scores = sorted(scores,reverse=True)
      candidates = scores[:top_k]
      print([i[0] for i in candidates])
      candidates = [(j,i) for i,j in candidates]
      candidates = sorted(candidates)
      if(candidates[0][0] <-4.8):
        row_dict[str(j)] = "This is Useless Content"
        continue
    
      candidates=[cont_l_1[i[0]] for i in candidates]
      # candidates=[candidates[i[1]] for i in candidates]
      row_dict[str(j)] = " ".join(candidates)
      deli.deli_2()
      deli.deli_2()
      printw("Selected Content "+str(j)+ ": "+" ".join(candidates) )
      # row_dict[str(j)] = " ".join(candidates)
      deli.deli_2()
      input()
      output.clear()
      
    df_ans  = df_ans.append(row_dict,ignore_index=True)
  print(make_histogram(len_array))
  return df_ans

def rank_sentences(q, passage, n_cls=7, select_cls=4):
  sent_l = nltk.sent_tokenize(passage)
  print("Total sentences:",len(sent_l))
  if(len(sent_l)==0 or (len(sent_l)==1 and sent_l[0]=='')):
    return "Insufficient String"
  if(len(sent_l)<n_cls): 
    return " ".join(passage)
  scores = []
  for i, k  in enumerate(sent_l):
    score = score_for_one(q,[k])
    scores.append((score,i))
  scores = sorted(scores,reverse=True)
  # print(scores)
  scoo = [i[0] for i in scores]
  x= np.array(scoo)
  km = KMeans(n_clusters=n_cls, random_state=42, max_iter=500)
  km.fit(x.reshape(-1,1))
  y = km.labels_
  sent_ll = [sent_l[iiii] for jjjj,iiii in scores]
  # print(list(zip(sent_ll,y)))
  candidates= []
  true_label = select_cls
  selecting = set()
  for ii,iii in enumerate(y):
    if(len(selecting)<select_cls):
      selecting.add(iii)
    if iii in selecting:
      candidates.append((scoo[ii], scores[ii][1]))
  print("Selected Sentences Count:",len(candidates))
  candidates = [(j,i) for i,j in candidates]
  candidates = sorted(candidates)  
  candidates=[sent_l[i[0]] for i in candidates]
  return " ".join(candidates)


def rank_passages_2(df_c,top_k=9):
  df_c=df_c.fillna("")
  df_ans = pd.DataFrame()
  # len_array=[]
  for i, k in df_c.iterrows():
    row_dict={}
    for j in range(10):
      print("Index",i,j)
      passage_ranker=[]
      q=df_c['query'].iloc[i]

      content= df_c[str(j) +"_Content"].iloc[i]
      # printw("Question "+str(i) +":" + q)
      # printw("Content "+str(j) +":" + df_c[str(j) +"_Link"].iloc[i])
      if(content==" this is a meaningless content due to repetition and timeout as well"):
        row_dict[str(j)] =" this is a meaningless content due to repetition and timeout as well"
        continue
      start = timeit.timeit()
      if i == 58 and j ==6:
        row_dict[str(j)] = " this is a meaningless content due to repetition and timeout as well"
        continue
      row_dict[str(j)] = rank_sentences(q,content)
      end = timeit.timeit()
      print(end-start)
      
    df_ans  = df_ans.append(row_dict,ignore_index=True)
  # print(make_histogram(len_array))
  return df_ans

In [ ]:
cont = '''

k-means clustering is a method of vector quantization, originally from signal processing, that aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid), serving as a prototype of the cluster. This results in a partitioning of the data space into Voronoi cells. k-means clustering minimizes within-cluster variances (squared Euclidean distances), but not regular Euclidean distances, which would be the more difficult Weber problem: the mean optimizes squared errors, whereas only the geometric median minimizes Euclidean distances. For instance, better Euclidean solutions can be found using k-medians and k-medoids.

The problem is computationally difficult (NP-hard); however, efficient heuristic algorithms converge quickly to a local optimum. These are usually similar to the expectation-maximization algorithm for mixtures of Gaussian distributions via an iterative refinement approach employed by both k-means and Gaussian mixture modeling. They both use cluster centers to model the data; however, k-means clustering tends to find clusters of comparable spatial extent, while the Gaussian mixture model allows clusters to have different shapes.

The unsupervised k-means algorithm has a loose relationship to the k-nearest neighbor classifier, a popular supervised machine learning technique for classification that is often confused with k-means due to the name. Applying the 1-nearest neighbor classifier to the cluster centers obtained by k-means classifies new data into the existing clusters. This is known as nearest centroid classifier or Rocchio algorithm.

The term "k-means" was first used by James MacQueen in 1967,[2] though the idea goes back to Hugo Steinhaus in 1956.[3] The standard algorithm was first proposed by Stuart Lloyd of Bell Labs in 1957 as a technique for pulse-code modulation, although it was not published as a journal article until 1982.[4] In 1965, Edward W. Forgy published essentially the same method, which is why it is sometimes referred to as the Lloyd–Forgy algorithm.[5]

Commonly used initialization methods are Forgy and Random Partition.[10] The Forgy method randomly chooses k observations from the dataset and uses these as the initial means. The Random Partition method first randomly assigns a cluster to each observation and then proceeds to the update step, thus computing the initial mean to be the centroid of the cluster's randomly assigned points. The Forgy method tends to spread the initial means out, while Random Partition places all of them close to the center of the data set. According to Hamerly et al.,[10] the Random Partition method is generally preferable for algorithms such as the k-harmonic means and fuzzy k-means. For expectation maximization and standard k-means algorithms, the Forgy method of initialization is preferable. A comprehensive study by Celebi et al.,[11] however, found that popular initialization methods such as Forgy, Random Partition, and Maximin often perform poorly, whereas Bradley and Fayyad's approach[12] performs "consistently" in "the best group" and k-means++ performs "generally well".


'''

query = "Is Kmeans computationally efficient?"
deli.deli_2()
ans = rank_sentences(query, cont, n_cls=7, select_cls =4)
print(ans)
print(get_answer_1(query,ans))

In [ ]:
def rank_pages(df_c):
  df_c= df_c.fillna("Insufficient String")
  list_1= []
  for i, k in df_c.iterrows():
    print(i)
    q = df_a['query'].iloc[i]
    temp_list = []
    for j in range(10):
      print("\t",j)
      try:
        c = k[str(j)]
        
        # print((c))
        try:
          # print("I am here")
          if(c == 'Insufficient String' or c ==' ' or c=='' ):
            continue
          print("\t\t",len(nltk.sent_tokenize(c)))
          ans, scores = get_answer_1(q,c)
          
          temp_list.append((harmonic_mean(scores), j, ans))
        except Exception as eeee:

          print("Error at stage 1:",eeee)

          c = rank_pass(q,c)
          print("\t\t After I have changed",len(nltk.sent_tokenize(c)))
          
          ans,scores = get_answer_1(q,c)
          temp_list.append((harmonic_mean(scores), j, ans))
      except:
        print("skipped bcz it took time")
    temp_list = sorted(temp_list, reverse=True)
    temp_list= temp_list[:3]
    # print(temp_list)
    temp_list = [ ii[2] for ii in temp_list]
    list_temp =[q]
    list_temp.extend(temp_list)
    list_1.append(list_temp)
  return list_1

##DPR


In [ ]:
#@title

torch.set_grad_enabled(False)
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

In [ ]:
#@title
model_dpr = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')


In [ ]:
#@title
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

In [ ]:
#@title
def get_dpr_similarity(ques, cont):
  
  ctx_tokens = ctx_tokenizer(cont, max_length = 512, truncation = True, return_tensors='pt')["input_ids"]
  q_tokens = q_tokenizer(ques,return_tensors='pt')["input_ids"]
  ctx_encoded = ctx_encoder(ctx_tokens).pooler_output
  q_encoded = q_encoder(q_tokens).pooler_output
  similarity = (q_encoded@(ctx_encoded.T)).item()

  return similarity

In [ ]:

def combine_strs(list_of_str):
  ans_list = []
  i=0
  while i < (len(list_of_str)):
    j = i
    temp_str =""
    while(j<len(list_of_str) and len(temp_str.split(" "))<256):
      temp_str+=list_of_str[j]+" "
      # print(temp_str)
      j+=1
    ans_list.append(temp_str)
    i=j
  return ans_list

def rank_pass(q,ct,alpha=0.5):
  cl = nltk.sent_tokenize(ct)
  
  if len(cl) > 3000:
    cl = combine_strs(cl)

  print(len(cl))
  similarity_scores = [get_dpr_similarity(q,a) for i,a in enumerate(cl)]
  # similarity_scores = sorted(similarity_scores, reverse = True)

  similarity_scores = np.array(similarity_scores)
  # print(similarity_scores)
  mean = np.mean(similarity_scores)
  std = np.std(similarity_scores) * alpha
  thresh1 = mean+std
  # s1 = np.where(similarity_scores >= thresh2,1,0)
  s2 = np.where(similarity_scores >= thresh1,1,0)
  s = np.where(s2 == 1)[0]
  # print(s)
  cl_list = [cl[j] for j in s]

  return " ".join(cl_list)

def rank_passages_3(df_c,top_k=9):
  df_c=df_c.fillna("")
  df_ans = pd.DataFrame()
  # len_array=[]
  for i, k in df_c.iterrows():
    row_dict={}
    for j in range(10):
      print("Index",i,j)
      passage_ranker=[]
      q=df_c['query'].iloc[i]

      content= df_c[str(j) +"_Content"].iloc[i]
      # printw("Question "+str(i) +":" + q)
      # printw("Content "+str(j) +":" + df_c[str(j) +"_Link"].iloc[i])
      if(content==" this is a meaningless content due to repetition and timeout as well"):
        row_dict[str(j)] =" this is a meaningless content due to repetition and timeout as well"
        continue
      start = timeit.timeit()
      row_dict[str(j)] = rank_pass(q,content)
      end = timeit.timeit()
      print(end-start)
      
    df_ans  = df_ans.append(row_dict,ignore_index=True)
  # print(make_histogram(len_array))
  return df_ans